In [5]:
import torch.nn as nn
import torch
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/home/kolek/Edu/project/video-to-3d/src/')
sys.path.insert(1, '/home/kolek/Edu/project/video-to-3d/src/pycolmap')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
nn.Sequential(nn.Linear(128, 128)) + nn.Sequential(nn.Linear(128, 128))

Sequential(
  (0): Linear(in_features=128, out_features=128, bias=True)
  (1): Linear(in_features=128, out_features=128, bias=True)
)

In [4]:
nn.Linear(129, 129) + nn.Sequential(nn.Linear(128, 128))

TypeError: unsupported operand type(s) for +: 'Linear' and 'Sequential'

In [5]:
a = torch.arange(129 * 3).reshape(3, 129)
a.shape


device = 'cuda'

In [9]:

import nerf_model

model = nerf_model.NerfModel(input_size_direction=3, input_size_ray=3).to(device)

batch_size = 5
n_rays = 20 * 20
n_samples = 20


# for each image in the batch we going to geneate 256 * 256 directions and rays


directions = torch.arange(batch_size * n_rays * n_samples * 3, dtype=torch.float32) \
                   .reshape(batch_size, n_rays, n_samples, 3).to(device)
rays = torch.arange(batch_size * n_rays * n_samples * 3, dtype=torch.float32) \
                   .reshape(batch_size, n_rays, n_samples, 3).to(device)



directions.shape
# batch_size n_rays, 4
model(rays, directions).shape

OutOfMemoryError: CUDA out of memory. Tried to allocate 40.00 MiB. GPU 0 has a total capacty of 3.81 GiB of which 28.06 MiB is free. Including non-PyTorch memory, this process has 3.78 GiB memory in use. Of the allocated memory 3.70 GiB is allocated by PyTorch, and 13.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [12]:
import pycolmap

scene_manager = pycolmap.SceneManager("/home/kolek/Edu/project/test_data/sparse/0")

In [21]:
scene_manager.load_cameras()
scene_manager.load_images()
len(scene_manager.cameras), len(scene_manager.images)

(1, 2)

In [29]:
camera = scene_manager.cameras[1]
print(camera)

OPENCV 960 1280 1083.3341134167667 1075.6291880913084 480.0 640.0 -0.021853695520177647 -0.022266433852242817 0.002635423093899853 -0.00126717372198474


In [57]:
import numpy as np

image = scene_manager.images[1]

focal_length_x = camera.fx
focal_length_y = camera.fy
principal_point_x = camera.cx
principal_point_y = camera.cy

# both camtopix and pixtocam are 3,3 matrix

camtopix = np.array([[focal_length_x, 0, principal_point_x],
                     [0, focal_length_y, principal_point_y],
                     [0, 0, 1]])
pixtocam = np.linalg.inv(camtopix)


# extract world2cam matrix from image

translation = image.C().reshape(3, 1)
rotation = image.R()
bottom = np.array([0, 0, 0, 1]).reshape(1, 4)
world2cam = np.concatenate([np.concatenate([rotation, translation], axis=1), bottom], axis=0)
cam2world = np.linalg.inv(world2cam)

#pose = cam2world[:3, :4]
#pose = cam2world
#pose @ np.diag([1, -1, -1, 1])

pixtocam

array([[ 9.23076258e-04,  0.00000000e+00, -4.43076604e-01],
       [ 0.00000000e+00,  9.29688420e-04, -5.95000589e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [80]:
from PIL import Image

img = Image.open(scene_manager.image_path + image.name)
img = img.resize([800, 800], Image.Resampling.LANCZOS)
img.mode
img.width

800

In [78]:
from torchvision import transforms

tr =  transforms.ToTensor()
img = tr(img)
img.shape

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [79]:
img.shape

torch.Size([3, 800, 800])